<a href="https://colab.research.google.com/github/newfull5/AI-Project/blob/master/summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch wandb tqdm evaluate rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


- model = t5-base
- dataset = CNN_dailymail 3.0.0
  - max_token_length = about 3,500
  

- config
  - max_length_truncate = 1024
  


In [2]:
from datasets import load_dataset
from tqdm import tqdm

In [3]:
# length 0 ~ 500 => 1900
# length 500 ~ 1000 => 5400
# length 1000 ~ 1500 => 2800
# length 1500 ~ 2000 -> 1150
# length 2000 ~  -> 500

In [4]:
import argparse

def _get_parser():  
    parser = argparse.ArgumentParser()
    parser.add_argument()
    return parser
   

args = argparse.Namespace(  
  model_name="t5-large", 
  tokenizer_name="t5-large",
  dataset_name=['cnn_dailymail', '3.0.0'],
  batch_size = 2,
  lr=3e-5,
  val_check_interval= 2000
)

In [5]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from datasets import load_dataset

class Dataset(Dataset):
  def __init__(self, args, stage):
    super().__init__()
    self.stage = stage
    self.args = args
    self.plain_target = []
    self.input_ids, self.attention_mask, self.decoder_input_ids, self.labels = self._get_data(args, stage)
    

  def _get_data(self, args, stage):
    dataset = load_dataset(args.dataset_name[0], args.dataset_name[1])
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name)
    input_ids = []
    attention_mask = []
    decoder_input_ids = []
    labels = []
    

    if stage in ['train', 'validation', 'test']:
      self.plain_target += dataset[stage]['highlights']
      for data_set in tqdm(dataset[stage]):
        inputs = tokenizer(
            text=data_set['article'],
            max_length=1024,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids.append(inputs['input_ids'])
        attention_mask.append(inputs['attention_mask'])
        
        outputs = tokenizer(
            text=data_set['highlights'],
            max_length=256,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        decoder_input_ids.append(outputs['input_ids'][:, :-1].contiguous())
        label = outputs['input_ids'][:, 1:].clone().detach()
        label[label == tokenizer.pad_token_id] = -100
        labels.append(label)

    else:
      raise Exception("you can set stage only 'train', 'test' or 'validation'")

    return input_ids, attention_mask, decoder_input_ids, labels
      
  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attention_mask[idx], self.decoder_input_ids[idx], self.labels[idx], self.plain_target[idx]

In [6]:
from transformers import AutoModelForSeq2SeqLM
import torch
from torch import nn

class Model(nn.Module):
  def __init__(self, args):
    super(Model, self).__init__()
    self.model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name)
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    self.model.to(self.device)

  def forward(self, batch):
    input_ids, attention_mask, decoder_input_ids, labels, _ = batch
    outputs = self.model(
        input_ids=self._move_to_cuda(input_ids).squeeze(),
        attention_mask=self._move_to_cuda(attention_mask),
        decoder_input_ids=self._move_to_cuda(decoder_input_ids).squeeze(),
        labels=self._move_to_cuda(labels)    
    )
    return outputs

  def _move_to_cuda(self, inputs):
    if torch.is_tensor(inputs):
      return inputs.to(self.device)
    elif isinstance(inputs, list):
      return [self._move_to_cuda(x) for x in inputs]
    elif isinstance(inputs, dict):
      return {key: self._move_to_cuda(value) for key, value in inputs.items()}
    else:
      return inputs

  def save(self, save_dir):
    self.model.save_pretrained(save_dir)

  def load(self, save_dir):
    self.model.load_state_dict(
        torch.load(f"{save_dir}/pytorch_model.bin", map_location=torch.device(self.device))
    )

In [7]:
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import wandb
import evaluate

class Trainer:
  def __init__(self, args, model, train_loader, validation_loader):
    self.args = args
    self.model = model
    self.train_loader = train_loader
    self.valid_loader = validation_loader
    self.global_steps = 0
    self.tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name)
    self.optimizer = torch.optim.SGD(self.model.parameters(), lr=args.lr)
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    self.val_check_interval = args.val_check_interval
    self.global_steps = 0
    self.rouge = evaluate.load('rouge')

  def training_phase(self):
    self.model.train()
    scaler = GradScaler()
    total_train_loss = 0
    train_steps = 0

    for batch in tqdm(self.train_loader):
      self.optimizer.zero_grad()
      with autocast():
        outputs = model(batch)
      
      scaler.scale(outputs.loss).backward()
      scaler.step(self.optimizer)
      scaler.update()
      self.global_steps += 1
      total_train_loss += float(outputs.loss)

      if self.global_steps % self.val_check_interval == 0 and self.global_steps != 0:
        wandb.log({
            'train_loss': (total_train_loss/self.val_check_interval)
            })
        
        total_train_loss = 0
        model.save(f"./cnn_daily_summrization/{self.global_steps}/")
        self.valid_phase()

  def valid_phase(self):
    self.model.eval()
    predictions = []
    total_valid_loss = 0

    for batch in self.valid_loader:
      outputs = model(batch)
      total_valid_loss += float(outputs.loss)
      predictions += [self.tokenizer.decode(logit.argmax(dim=1)).strip() for logit in outputs.logits]
    
    rouge_score = self.rouge.compute(predictions=predictions,
                                    references=[t[-1] for t in self.valid_loader],
                                    tokenizer=lambda x: self.tokenizer(x)['input_ids'])
    
    wandb.log({
        'rouge1': rouge_score['rouge1'],
        'rouge2': rouge_score['rouge2'],
        'rougeL': rouge_score['rougeL'],
        'rougeLsum': rouge_score['rougeLsum'],
        'valid_loss': total_valid_loss/(len(self.valid_loader//len(self.args.batch_size)))
    })

  def fit(self):
    self.training_phase()

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset = Dataset(args, 'train'),
    batch_size = args.batch_size,
    shuffle = True
)

valid_loader = DataLoader(
    dataset = Dataset(args, 'validation'),
    batch_size = args.batch_size,
    shuffle = True
)

model = Model(args)

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
  3%|▎         | 9525/287113 [00:27<12:59, 356.05it/s]

In [ ]:
trainer = Trainer(
    args=args,
    model=model,
    train_loader=train_loader,
    validation_loader=valid_loader
)

In [ ]:
!nvidia-smi

In [ ]:
wandb.init('Summarization')

trainer.fit()